## Importazione librerie

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle
import numpy as np

In [4]:
file_path = "../../train.csv"
df = pd.read_csv(file_path)

## Controllo ed eliminazione valori nulli e duplicati

In [6]:
print("\nValori mancanti nel DataFrame:")
print(df.isnull().sum())


Valori nulli nel DataFrame:
Year    0
S0      0
S1      0
S2      0
S3      0
       ..
S85     0
S86     0
S87     0
S88     0
S89     0
Length: 91, dtype: int64


In [5]:
# Gestione dei duplicati
print("\nDuplicati nel DataFrame:")
print(df.duplicated().sum())


Duplicati nel DataFrame:
52


In [ ]:
df2 = df.dropna()
df2 = df.drop_duplicates()